In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
from huggingface_hub import notebook_login
notebook_login()

In [33]:
# import pandas as pd

# train_path = r"C:\Users\bhave\Documents\Genesys Feb 2025\Trial Data"
# df = pd.read_csv(r"C:\Users\bhave\Documents\Genesys Feb 2025\Trial Data\trial.csv")


In [34]:
# import pandas as pd
# import os
# import librosa
# from datasets import Dataset, DatasetDict
# from sklearn.model_selection import train_test_split


# def load_audio(file_path):
#     # Load audio file using librosa
#     audio, sample_rate = librosa.load(file_path, sr=16000)
#     return audio

# # Example: Load an audio file
# sample_audio = load_audio(df['file_path'][0])
# print(f"Audio sample shape: {sample_audio.shape}")


In [35]:
# # Load all audio files and transcriptions
# audio_files = [load_audio(file) for file in df['file_path']]
# transcriptions = df['transcription'].tolist()

# # Create the dataset dictionary
# data_dict = {'audio': audio_files, 'sentence': transcriptions}

# # Convert to a HuggingFace Dataset
# dataset = Dataset.from_dict(data_dict)

# # Create the DatasetDict
# dataset_dict = DatasetDict({
#     'train': dataset
# })

# # Check the resulting dataset
# print(dataset_dict)


In [36]:
# import pandas as pd
# import librosa
# from datasets import Dataset, DatasetDict


# def load_audio(file_path):
#     # Load audio file using librosa
#     audio, sample_rate = librosa.load(file_path, sr=16000)  # Ensure sample rate is 16000
#     return {'array': audio, 'sampling_rate': sample_rate}

# # Load all audio files and transcriptions
# audio_files = [load_audio(file) for file in df['file_path']]
# transcriptions = df['transcription'].tolist()

# # Create the dataset dictionary
# data_dict = {'audio': audio_files, 'sentence': transcriptions}

# # Convert to a HuggingFace Dataset
# dataset = Dataset.from_dict(data_dict)

# # Split into train and test datasets using Hugging Face's train_test_split method
# dataset = dataset.train_test_split(test_size=0.2)

# # The resulting dataset will have 'train' and 'test' keys
# dataset_dict = DatasetDict({
#     'train': dataset['train'],
#     'test': dataset['test']
# })

# # Check the resulting dataset
# print(dataset_dict)


In [37]:
# # Specify the directory where you want to save the dataset
# output_dir = r"C:\Users\bhave\Documents\Genesys Feb 2025"  # Replace with your desired path

# # Save the dataset to disk
# dataset_dict.save_to_disk(output_dir)


In [38]:
# def prepare_dataset(batch):
#     # load and resample audio data from 48 to 16kHz
#     audio = batch["audio"]

#     # compute log-Mel input features from input audio array
#     batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

#     # encode target text to label ids
#     batch["labels"] = tokenizer(batch["sentence"]).input_ids
#     return batch



from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

# Initialize Whisper feature extractor and tokenizer
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

# Create a processor (optional, if you'd like to use a processor that combines both)
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")







In [39]:
def prepare_dataset(batch, feature_extractor, tokenizer):
    input_features = []
    labels = []
    attention_masks = []

    for audio, sentence in zip(batch["audio"], batch["sentence"]):
        # Process audio
        audio_array = audio['array']
        sampling_rate = audio['sampling_rate']
        input_feature = feature_extractor(audio_array, sampling_rate=sampling_rate).input_features[0]
        input_features.append(input_feature)

        # Process text with attention mask
        tokenized = tokenizer(sentence, return_attention_mask=True)
        labels.append(tokenized.input_ids)
        attention_masks.append(tokenized.attention_mask)

    # Add processed features and attention masks
    batch["input_features"] = input_features
    batch["labels"] = labels
    batch["attention_mask"] = attention_masks
    return batch


In [40]:
%pip install datasets

In [41]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor
from datasets import load_from_disk

# Initialize Whisper feature extractor and tokenizer
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

# Create a processor (optional, if you'd like to use a processor that combines both)
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")



def prepare_dataset(batch, feature_extractor, tokenizer):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


# Load your dataset (make sure it's correctly loaded)
dataset_path = r"/content/drive/MyDrive/500data"
dataset_dict = load_from_disk(dataset_path)

print("dataset_dict: ")
print(dataset_dict)
print(type(dataset_dict))

# Manually iterate through the dataset and apply the prepare_dataset function
processed_data = []

# Iterate through the dataset (assuming it's split into "train" or other splits, like in the case of HuggingFace datasets)
for split_name, split_data in dataset_dict.items():
    print(f"Processing split: {split_name}")

    # Prepare each batch in the split
    processed_split_data = []
    for batch in split_data:
        processed_batch = prepare_dataset(batch, feature_extractor, tokenizer)
        processed_split_data.append(processed_batch)

    # Add the processed split to the final dataset
    processed_data.append((split_name, processed_split_data))

# Optionally: Combine processed data into a dataset-like object, depending on your use case
# If you want to convert processed data back to a DatasetDict, you can do so like this:
from datasets import DatasetDict

final_dataset_dict = DatasetDict({
    split_name: processed_split_data for split_name, processed_split_data in processed_data
})



dataset_dict: 
DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 400
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 100
    })
})
<class 'datasets.dataset_dict.DatasetDict'>
Processing split: train
Processing split: test


In [42]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [43]:
model.generation_config.language = "English"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [44]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [45]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [46]:
%pip install evaluate jiwer

In [47]:
import evaluate

metric = evaluate.load("wer")

In [48]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [59]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./500medimix",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-4,
    warmup_steps=50,
    num_train_epochs=5,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="epoch",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_strategy="epoch",
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=final_dataset_dict["train"],
    eval_dataset=final_dataset_dict["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-59-e9d2e434161f>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [60]:
processor.save_pretrained(training_args.output_dir)

[]

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Wer
1,0.348700,0.347463,11.037037
2,0.109300,0.468434,12.740741


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

In [ ]:
kwargs = {
    "dataset": "500 pes_colab",
    "dataset_args": "config: English, split: test",
    "language": "en",
    "model_name": "Whisper Small En Medimix",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
trainer.push_to_hub(**kwargs)

In [ ]:
# from transformers import pipeline
# import gradio as gr

# pipe = pipeline(model="Bhaveen/500medimix")  # change to "your-username/the-name-you-picked"

# def transcribe(audio):
#     text = pipe(audio)["text"]
#     return text

# iface = gr.Interface(
#     fn=transcribe,
#     inputs=gr.Audio(source="microphone", type="filepath"),
#     outputs="text",
#     title="Whisper Small English",
#     description="Realtime demo for English speech recognition using a fine-tuned Whisper small model.",
# )

# iface.launch()